# Levenberg-Marquardt Algorithm

The Levenberg-Marquardt (LM) algorithm is a powerful and widely used optimization technique for solving non-linear least squares problems. It cleverly combines the strengths of two other popular optimization methods: the **Gradient Descent (or Steepest Descent) method** and the **Gauss-Newton method**. This hybrid approach gives LM its robustness and efficient convergence properties.

## Understanding the Problem: Non-Linear Least Squares

Imagine you have a set of observed data points $(x_i, y_i)$ and you want to fit a model function $f(x, \mathbf{\beta})$ to this data, where $\mathbf{\beta}$ is a vector of unknown parameters. The function $f$ is *non-linear* with respect to these parameters. Our goal is to find the values of $\mathbf{\beta}$ that minimize the difference between the observed $y_i$ values and the predicted $f(x_i, \mathbf{\beta})$ values.

The "least squares" part means we want to minimize the sum of the squares of the residuals (errors). A residual for a single data point $(x_i, y_i)$ is defined as:
$r_i(\mathbf{\beta}) = y_i - f(x_i, \mathbf{\beta})$

We want to minimize the objective function $S(\mathbf{\beta})$, which is the sum of squared residuals:
$$S(\mathbf{\beta}) = \sum_{i=1}^m [y_i - f(x_i, \mathbf{\beta})]^2 = \sum_{i=1}^m r_i(\mathbf{\beta})^2$$
where $m$ is the number of data points.

In vector notation, if $\mathbf{r}(\mathbf{\beta})$ is a vector of all residuals $r_i(\mathbf{\beta})$, then:
$$S(\mathbf{\beta}) = \mathbf{r}(\mathbf{\beta})^T \mathbf{r}(\mathbf{\beta}) = ||\mathbf{r}(\mathbf{\beta})||^2$$

## The Two Parents of Levenberg-Marquardt

To understand LM, let's briefly look at its "parents":

### Gradient Descent (Steepest Descent) Method

Gradient Descent is a first-order optimization algorithm. It works by iteratively taking steps in the direction opposite to the gradient of the objective function. The idea is that the negative gradient points in the direction of the steepest decrease of the function.

To minimize $S(\mathbf{\beta})$, we update $\mathbf{\beta}$ as:
$$\mathbf{\beta}_{k+1} = \mathbf{\beta}_k - \alpha \nabla S(\mathbf{\beta}_k)$$
where $\alpha$ is the learning rate (step size), and $\nabla S(\mathbf{\beta}_k)$ is the gradient of $S(\mathbf{\beta})$ at $\mathbf{\beta}_k$.

Let's derive the gradient $\nabla S(\mathbf{\beta})$:

$$S(\mathbf{\beta}) = \sum_{i=1}^m r_i(\mathbf{\beta})^2$$

The partial derivative with respect to a parameter $\beta_j$ is:

$$\frac{\partial S}{\partial \beta_j} = \sum_{i=1}^m 2 r_i(\mathbf{\beta}) \frac{\partial r_i}{\partial \beta_j}$$

Recall $r_i(\mathbf{\beta}) = y_i - f(x_i, \mathbf{\beta})$, so $\frac{\partial r_i}{\partial \beta_j} = - \frac{\partial f(x_i, \mathbf{\beta})}{\partial \beta_j}$.

Therefore,

$$\frac{\partial S}{\partial \beta_j} = \sum_{i=1}^m -2 r_i(\mathbf{\beta}) \frac{\partial f(x_i, \mathbf{\beta})}{\partial \beta_j}$$

In vector form, the gradient $\nabla S(\mathbf{\beta})$ can be written as:$$\nabla S(\mathbf{\beta}) = -2 \mathbf{J}(\mathbf{\beta})^T \mathbf{r}(\mathbf{\beta})$$

where $\mathbf{J}(\mathbf{\beta})$ is the Jacobian matrix of the residual vector $\mathbf{r}(\mathbf{\beta})$ with respect to $\mathbf{\beta}$. 

The element $J_{ij}$ of the Jacobian matrix is given by:

$$J_{ij}(\mathbf{\beta}) = \frac{\partial r_i(\mathbf{\beta})}{\partial \beta_j} = - \frac{\partial f(x_i, \mathbf{\beta})}{\partial \beta_j}$$

So, the Gradient Descent update rule is:

$$\mathbf{\beta}_{k+1} = \mathbf{\beta}_k + 2\alpha \mathbf{J}(\mathbf{\beta}_k)^T \mathbf{r}(\mathbf{\beta}_k)$$

**Strengths:** Simple to implement, guaranteed to converge (eventually) to a local minimum if $\alpha$ is small enough.
**Weaknesses:** Can be very slow to converge, especially in narrow valleys or when far from the minimum.

### Gauss-Newton Method

The Gauss-Newton method is a second-order optimization algorithm (though it doesn't explicitly compute the full Hessian). It approximates the objective function locally with a quadratic model and then finds the minimum of that quadratic. It's particularly well-suited for least squares problems because it exploits the structure of the sum of squares.

We want to find a step $\mathbf{\delta}$ such that $\mathbf{\beta}_{k+1} = \mathbf{\beta}_k + \mathbf{\delta}$ minimizes $S(\mathbf{\beta}_k + \mathbf{\delta})$.
Let's approximate $f(x_i, \mathbf{\beta}_k + \mathbf{\delta})$ using a first-order Taylor expansion around $\mathbf{\beta}_k$:
$$f(x_i, \mathbf{\beta}_k + \mathbf{\delta}) \approx f(x_i, \mathbf{\beta}_k) + \sum_{j=1}^n \frac{\partial f(x_i, \mathbf{\beta}_k)}{\partial \beta_j} \delta_j$$
where $n$ is the number of parameters in $\mathbf{\beta}$.

This can be written in vector form as:

$$\mathbf{f}(\mathbf{\beta}_k + \mathbf{\delta}) \approx \mathbf{f}(\mathbf{\beta}_k) + \mathbf{J}_f(\mathbf{\beta}_k) \mathbf{\delta}$$

where $\mathbf{J}_f(\mathbf{\beta}_k)$ is the Jacobian of the model function $\mathbf{f}$ (not the residuals).

So, the residual vector becomes:

$$\mathbf{r}(\mathbf{\beta}_k + \mathbf{\delta}) = \mathbf{y} - \mathbf{f}(\mathbf{\beta}_k + \mathbf{\delta}) \approx \mathbf{y} - (\mathbf{f}(\mathbf{\beta}_k) + \mathbf{J}_f(\mathbf{\beta}_k) \mathbf{\delta})$$

Let $\mathbf{r}_k = \mathbf{y} - \mathbf{f}(\mathbf{\beta}_k)$. Then:

$$\mathbf{r}(\mathbf{\beta}_k + \mathbf{\delta}) \approx \mathbf{r}_k - \mathbf{J}_f(\mathbf{\beta}_k) \mathbf{\delta}$$

The objective function to minimize with respect to $\mathbf{\delta}$ is now:

$$S(\mathbf{\beta}_k + \mathbf{\delta}) \approx ||\mathbf{r}_k - \mathbf{J}_f(\mathbf{\beta}_k) \mathbf{\delta}||^2$$

Let $\mathbf{J} = \mathbf{J}_f(\mathbf{\beta}_k)$. We want to minimize $||\mathbf{r}_k - \mathbf{J}\mathbf{\delta}||^2$.

This is a linear least squares problem in terms of $\mathbf{\delta}$.

To find the minimum, we take the derivative with respect to $\mathbf{\delta}$ and set it to zero.

$$||\mathbf{r}_k - \mathbf{J}\mathbf{\delta}||^2 = (\mathbf{r}_k - \mathbf{J}\mathbf{\delta})^T (\mathbf{r}_k - \mathbf{J}\mathbf{\delta})$$

$$= \mathbf{r}_k^T \mathbf{r}_k - \mathbf{r}_k^T \mathbf{J}\mathbf{\delta} - (\mathbf{J}\mathbf{\delta})^T \mathbf{r}_k + (\mathbf{J}\mathbf{\delta})^T (\mathbf{J}\mathbf{\delta})$$

$$= \mathbf{r}_k^T \mathbf{r}_k - 2\mathbf{\delta}^T \mathbf{J}^T \mathbf{r}_k + \mathbf{\delta}^T \mathbf{J}^T \mathbf{J}\mathbf{\delta}$$

Taking the gradient with respect to $\mathbf{\delta}$:$$\nabla_{\mathbf{\delta}} S = -2 \mathbf{J}^T \mathbf{r}_k + 2 \mathbf{J}^T \mathbf{J}\mathbf{\delta}$$Setting to zero:

$$-2 \mathbf{J}^T \mathbf{r}_k + 2 \mathbf{J}^T \mathbf{J}\mathbf{\delta} = \mathbf{0}$$

$$\mathbf{J}^T \mathbf{J}\mathbf{\delta} = \mathbf{J}^T \mathbf{r}_k$$

This is the **normal equation** for the linear least squares problem.

The step $\mathbf{\delta}_{GN}$ is then:

$$\mathbf{\delta}_{GN} = (\mathbf{J}^T \mathbf{J})^{-1} \mathbf{J}^T \mathbf{r}_k$$

And the update rule is $\mathbf{\beta}_{k+1} = \mathbf{\beta}_k + \mathbf{\delta}_{GN}$.

Note that $\mathbf{J}^T \mathbf{J}$ is an approximation of the Hessian matrix of $S(\mathbf{\beta})$. Specifically, the Hessian $\mathbf{H}$ of $S(\mathbf{\beta})$ is given by $\mathbf{H} = 2\mathbf{J}^T \mathbf{J} + 2\sum_{i=1}^m r_i(\mathbf{\beta}) \nabla^2 r_i(\mathbf{\beta})$. The Gauss-Newton method ignores the second term, which is often small, especially when residuals are small or the model is "nearly linear."

**Strengths:** Can converge very fast when close to the minimum, as it approximates a second-order method.
**Weaknesses:** May diverge if the initial guess is far from the minimum, especially if $\mathbf{J}^T \mathbf{J}$ is singular or ill-conditioned (i.e., not invertible).

## The Levenberg-Marquardt Algorithm: The Best of Both Worlds

The Levenberg-Marquardt algorithm interpolates between the Gradient Descent and Gauss-Newton methods. It introduces a "damping parameter," $\lambda$ (lambda), which controls this interpolation.

The LM update equation for the step $\mathbf{\delta}_{LM}$ is:

$$(\mathbf{J}^T \mathbf{J} + \lambda \mathbf{D}) \mathbf{\delta}_{LM} = \mathbf{J}^T \mathbf{r}$$

where $\mathbf{D}$ is a diagonal matrix.

Historically, $\mathbf{D}$ was often the identity matrix $\mathbf{I}$, leading to the equation:

$$(\mathbf{J}^T \mathbf{J} + \lambda \mathbf{I}) \mathbf{\delta}_{LM} = \mathbf{J}^T \mathbf{r}$$

However, a more common and robust choice for $\mathbf{D}$ is the diagonal matrix containing the diagonal elements of $\mathbf{J}^T \mathbf{J}$. This scaling helps to make the problem better conditioned. Let's denote this by $\text{diag}(\mathbf{J}^T \mathbf{J})$.

So, the LM equation for the step $\mathbf{\delta}$ is:

$$(\mathbf{J}^T \mathbf{J} + \lambda \text{diag}(\mathbf{J}^T \mathbf{J})) \mathbf{\delta} = \mathbf{J}^T \mathbf{r}$$

Let's analyze the behavior based on $\lambda$:
* **When $\lambda$ is small ($\lambda \to 0$):** The term $\lambda \text{diag}(\mathbf{J}^T \mathbf{J})$ becomes negligible. The equation approaches the Gauss-Newton equation:

    $$(\mathbf{J}^T \mathbf{J}) \mathbf{\delta} \approx \mathbf{J}^T \mathbf{r}$$

    This means LM behaves like Gauss-Newton, offering fast convergence when near the minimum.

* **When $\lambda$ is large ($\lambda \to \infty$):** The term $\mathbf{J}^T \mathbf{J}$ becomes negligible compared to $\lambda \text{diag}(\mathbf{J}^T \mathbf{J})$. The equation approaches:

    $$\lambda \text{diag}(\mathbf{J}^T \mathbf{J}) \mathbf{\delta} \approx \mathbf{J}^T \mathbf{r}$$

    $$\mathbf{\delta} \approx \frac{1}{\lambda} (\text{diag}(\mathbf{J}^T \mathbf{J}))^{-1} \mathbf{J}^T \mathbf{r}$$

    This step is in a similar direction to the steepest descent direction. To see this, recall that the gradient descent direction is proportional to $\mathbf{J}^T \mathbf{r}$. The damping term ensures that the step size is small, preventing divergence when far from the minimum. The scaling by $\text{diag}(\mathbf{J}^T \mathbf{J})$ further aligns it with a scaled steepest descent, addressing potential scaling issues in the parameters.

**The key idea of LM is to adaptively adjust $\lambda$ at each iteration:**
* If a step leads to a *reduction* in the sum of squares $S(\mathbf{\beta})$, the step is accepted, and $\lambda$ is *decreased* for the next iteration. This makes the algorithm lean more towards Gauss-Newton, aiming for faster convergence.
* If a step leads to an *increase* in $S(\mathbf{\beta})$, the step is rejected, and $\lambda$ is *increased*. This makes the algorithm lean more towards Gradient Descent, taking smaller, more conservative steps to ensure progress.



## Derivation of the Levenberg-Marquardt Equation

The LM algorithm can be derived from a "trust-region" perspective or by directly combining the ideas of Gauss-Newton and Gradient Descent. Let's follow a derivation that highlights its connection to both.

We want to find a step $\mathbf{\delta}$ that minimizes $S(\mathbf{\beta}_k + \mathbf{\delta})$. We use the quadratic approximation of $S(\mathbf{\beta})$ around $\mathbf{\beta}_k$:
$$S(\mathbf{\beta}_k + \mathbf{\delta}) \approx S(\mathbf{\beta}_k) + \nabla S(\mathbf{\beta}_k)^T \mathbf{\delta} + \frac{1}{2} \mathbf{\delta}^T \mathbf{H}(\mathbf{\beta}_k) \mathbf{\delta}$$
where $\mathbf{H}(\mathbf{\beta}_k)$ is the Hessian matrix of $S(\mathbf{\beta})$.

As seen earlier, $\nabla S(\mathbf{\beta}) = -2 \mathbf{J}(\mathbf{\beta})^T \mathbf{r}(\mathbf{\beta})$.
The Hessian $\mathbf{H}(\mathbf{\beta})$ is given by:
$$\mathbf{H}(\mathbf{\beta}) = \frac{\partial^2 S}{\partial \beta_j \partial \beta_k} = 2 \mathbf{J}(\mathbf{\beta})^T \mathbf{J}(\mathbf{\beta}) + 2 \sum_{i=1}^m r_i(\mathbf{\beta}) \frac{\partial^2 r_i(\mathbf{\beta})}{\partial \beta_j \partial \beta_k}$$
The Gauss-Newton method approximates the Hessian by ignoring the second term, assuming residuals $r_i$ are small or the second derivatives $\frac{\partial^2 r_i}{\partial \beta_j \partial \beta_k}$ are negligible. So, $\mathbf{H}_{GN} = 2 \mathbf{J}^T \mathbf{J}$.

Substituting these into the quadratic approximation (and dividing by 2 for convenience in the minimization):
We want to minimize $S(\mathbf{\beta}_k) - (\mathbf{J}^T \mathbf{r})_k^T \mathbf{\delta} + \frac{1}{2} \mathbf{\delta}^T (\mathbf{J}^T \mathbf{J})_k \mathbf{\delta}$. (Here, subscripts $k$ denote evaluation at $\mathbf{\beta}_k$).

Taking the gradient with respect to $\mathbf{\delta}$ and setting to zero:

$$-(\mathbf{J}^T \mathbf{r})_k + (\mathbf{J}^T \mathbf{J})_k \mathbf{\delta} = \mathbf{0}$$

$$(\mathbf{J}^T \mathbf{J})_k \mathbf{\delta} = (\mathbf{J}^T \mathbf{r})_k$$
This is the Gauss-Newton step.

The Levenberg-Marquardt modification adds a damping term to this equation. It can be seen as minimizing the quadratic approximation of $S(\mathbf{\beta}_k + \mathbf{\delta})$ subject to a trust-region constraint, or equivalently, as a regularization technique. The objective becomes:

$$\min_{\mathbf{\delta}} \left( S(\mathbf{\beta}_k) - (\mathbf{J}^T \mathbf{r})_k^T \mathbf{\delta} + \frac{1}{2} \mathbf{\delta}^T (\mathbf{J}^T \mathbf{J})_k \mathbf{\delta} + \frac{\lambda}{2} ||\mathbf{\delta}||^2 \right)$$

The term $\frac{\lambda}{2} ||\mathbf{\delta}||^2$ (or $\frac{\lambda}{2} \mathbf{\delta}^T \mathbf{D} \mathbf{\delta}$) is added to penalize large step sizes, making the algorithm more stable.

Taking the gradient with respect to $\mathbf{\delta}$ and setting to zero:

$$-(\mathbf{J}^T \mathbf{r})_k + (\mathbf{J}^T \mathbf{J})_k \mathbf{\delta} + \lambda \mathbf{\delta} = \mathbf{0}$$

$$(\mathbf{J}^T \mathbf{J})_k \mathbf{\delta} + \lambda \mathbf{I} \mathbf{\delta} = (\mathbf{J}^T \mathbf{r})_k$$

$$(\mathbf{J}^T \mathbf{J} + \lambda \mathbf{I}) \mathbf{\delta} = \mathbf{J}^T \mathbf{r}$$

This is the original Levenberg-Marquardt equation with $\mathbf{D} = \mathbf{I}$.

If we use $\mathbf{D} = \text{diag}(\mathbf{J}^T \mathbf{J})$, the equation becomes:

$$(\mathbf{J}^T \mathbf{J} + \lambda \text{diag}(\mathbf{J}^T \mathbf{J})) \mathbf{\delta} = \mathbf{J}^T \mathbf{r}$$

This linear system is then solved for $\mathbf{\delta}$ at each iteration.

## The Algorithm Steps

1.  **Initialize:**
    * Choose an initial guess for the parameter vector $\mathbf{\beta}_0$.
    * Set an initial damping parameter $\lambda_0$ (e.g., $0.001$).
    * Set factors for increasing and decreasing $\lambda$ (e.g., $\nu_{inc} = 10$, $\nu_{dec} = 2$ or $3$).
    * Set convergence criteria (e.g., tolerance for change in $\mathbf{\beta}$, tolerance for change in $S$, maximum iterations).

2.  **Iterate (for k = 0, 1, 2, ...):**
    a.  **Calculate Residuals and Jacobian:**
        * Evaluate the residual vector $\mathbf{r}(\mathbf{\beta}_k) = \mathbf{y} - \mathbf{f}(\mathbf{\beta}_k)$.
        * Calculate the Jacobian matrix $\mathbf{J}(\mathbf{\beta}_k)$ where $J_{ij} = -\frac{\partial f(x_i, \mathbf{\beta}_k)}{\partial \beta_j}$.
        * Compute $S(\mathbf{\beta}_k) = ||\mathbf{r}(\mathbf{\beta}_k)||^2$.

    b.  **Construct and Solve the LM System:**
        * Form the matrix $\mathbf{A} = \mathbf{J}(\mathbf{\beta}_k)^T \mathbf{J}(\mathbf{\beta}_k) + \lambda_k \text{diag}(\mathbf{J}(\mathbf{\beta}_k)^T \mathbf{J}(\mathbf{\beta}_k))$.
        * Form the vector $\mathbf{g} = \mathbf{J}(\mathbf{\beta}_k)^T \mathbf{r}(\mathbf{\beta}_k)$.
        * Solve the linear system $\mathbf{A} \mathbf{\delta} = \mathbf{g}$ for $\mathbf{\delta}$.

    c.  **Evaluate New Parameters and Objective:**
        * Propose new parameters: $\mathbf{\beta}_{new} = \mathbf{\beta}_k + \mathbf{\delta}$.
        * Calculate the new sum of squares: $S(\mathbf{\beta}_{new}) = ||\mathbf{r}(\mathbf{\beta}_{new})||^2$.

    d.  **Update $\lambda$ and $\mathbf{\beta}$:**
        * **If $S(\mathbf{\beta}_{new}) < S(\mathbf{\beta}_k)$ (improvement):**
            * Accept the new parameters: $\mathbf{\beta}_{k+1} = \mathbf{\beta}_{new}$.
            * Decrease $\lambda$: $\lambda_{k+1} = \lambda_k / \nu_{dec}$.
        * **If $S(\mathbf{\beta}_{new}) \ge S(\mathbf{\beta}_k)$ (no improvement or worse):**
            * Reject the step: $\mathbf{\beta}_{k+1} = \mathbf{\beta}_k$.
            * Increase $\lambda$: $\lambda_{k+1} = \lambda_k \times \nu_{inc}$.
            * Go back to step 2b (with the increased $\lambda$ and the same $\mathbf{\beta}_k$).

    e.  **Check for Convergence:**
        * If the change in $\mathbf{\beta}$ or $S$ is below a predefined tolerance, or max iterations reached, stop.

## When can it be used? (Applications)

The Levenberg-Marquardt algorithm is a go-to method for any problem that can be formulated as a **non-linear least squares minimization**. Its robustness makes it widely applicable in various fields:

* **Curve Fitting / Data Fitting:** This is the most common application.
    * Fitting experimental data to a non-linear model (e.g., exponential decays, sigmoid functions, Gaussian peaks).
    * Determining parameters of physical, chemical, or biological models from observed data.
* **Computer Vision:**
    * **Camera Calibration:** Estimating intrinsic and extrinsic parameters of a camera from known 3D points and their 2D projections.
    * **3D Reconstruction:** Reconstructing 3D scenes or objects from multiple 2D images, including bundle adjustment (jointly optimizing camera poses and 3D point locations).
    * **Image Registration:** Aligning different images of the same scene.
* **Machine Learning / Neural Networks:**
    * **Training Neural Networks:** While Adam or SGD are more common for very large networks, LM can be highly effective for training smaller to medium-sized neural networks, especially when high precision is required, as it combines the speed of Gauss-Newton near the minimum with the stability of gradient descent further away. It's often faster and more reliable than basic backpropagation.
* **Robotics:**
    * **Inverse Kinematics:** Finding the joint angles of a robot arm to reach a desired end-effector position.
    * **Localization and Mapping (SLAM):** Optimizing robot pose and map features based on sensor readings.
* **Signal Processing:**
    * **System Identification:** Estimating parameters of a system model from input-output data.
    * **Filter Design:** Optimizing filter coefficients to match a desired response.
* **Chemistry and Physics:**
    * **Spectroscopy:** Analyzing spectral data to determine concentrations of substances or molecular parameters.
    * **Crystallography:** Refining crystal structures from diffraction data.
    * **Pharmacokinetics:** Modeling drug concentration in the body over time.
* **Econometrics:**
    * Estimating parameters in non-linear economic models.

### Advantages of Levenberg-Marquardt:

* **Robustness:** Combines the strengths of Gradient Descent (reliable far from minimum) and Gauss-Newton (fast near minimum). It tends to converge even with poor initial guesses.
* **Efficiency:** Often converges faster than first-order methods like Gradient Descent, especially near the minimum.
* **No line search needed:** The damping parameter adaptively controls the step size, often eliminating the need for complex line search procedures.

### Limitations:

* **Local Minima:** Like most iterative optimization algorithms, LM can only guarantee convergence to a local minimum, not necessarily the global minimum. The quality of the initial guess is still important.
* **Computational Cost:** Requires computing and inverting (or solving a linear system with) the matrix $\mathbf{J}^T \mathbf{J} + \lambda \text{diag}(\mathbf{J}^T \mathbf{J})$ at each iteration. This involves matrix multiplication and inversion, which can be computationally expensive for problems with a very large number of parameters (large $n$).
* **Memory Requirement:** Storing the Jacobian matrix can be memory-intensive for problems with many data points and/or many parameters.

In summary, the Levenberg-Marquardt algorithm is a powerful and popular choice for non-linear least squares problems due to its balanced approach that leverages the best features of both gradient descent and Gauss-Newton methods.

## Additional Materials

* https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm